# Setting base configs

Setting the default layer of the notebook to Bronze, importing libraries and setting main path

In [0]:
import pyspark.sql.functions as f
import pyspark.sql.types as t

main_path = "dbfs:/mnt/borges_portifolio"

spark.catalog.setCurrentDatabase('BRONZE')

#Reviewing previous data to plan the next steps

In [0]:
%sql
DESCRIBE bronze

col_name,data_type,comment
key,binary,null
value,binary,null
topic,string,null
partition,bigint,null
offset,bigint,null
timestamp,bigint,null
date,date,null
week_part,string,null
# Partition Information,,
# col_name,data_type,comment


In [0]:
batch_df = spark.table("bronze")
display(batch_df.limit(10))

key,value,topic,partition,offset,timestamp,date,week_part
MTE5NzE1,eyJkZXZpY2VfaWQiOiAxMTk3MTUsICJ0aW1lIjogMTU3NTE1ODQwMCwgImhlYXJ0cmF0ZSI6IDY2LjYyMjQ0OTczMTU3OTc2fQ==,bpm,0,65087128,1575158410818,2019-12-01,2019-48
MTA3OTUy,eyJkZXZpY2VfaWQiOiAxMDc5NTIsICJ0aW1lIjogMTU3NTE1ODQwMSwgImhlYXJ0cmF0ZSI6IDExNi4wMzEyNTg5ODAxMjA4N30=,bpm,2,65102257,1575158411394,2019-12-01,2019-48
MTYxOTEw,eyJkZXZpY2VfaWQiOiAxNjE5MTAsICJ0aW1lIjogMTU3NTE1ODQwMywgImhlYXJ0cmF0ZSI6IDgyLjMzMzg1OTEyMDg3MzE3fQ==,bpm,0,65087131,1575158417678,2019-12-01,2019-48
MTE2NzA4,eyJkZXZpY2VfaWQiOiAxMTY3MDgsICJ0aW1lIjogMTU3NTE1ODQxMiwgImhlYXJ0cmF0ZSI6IDgzLjEzNDUyNzA2MDg3NzA5fQ==,bpm,3,64899138,1575158421868,2019-12-01,2019-48
MTYxMzQz,eyJkZXZpY2VfaWQiOiAxNjEzNDMsICJ0aW1lIjogMTU3NTE1ODQyMCwgImhlYXJ0cmF0ZSI6IDU5LjcxODMyNzY1MTYyODA0fQ==,bpm,3,64899148,1575158430841,2019-12-01,2019-48
MTEzMDI0,eyJkZXZpY2VfaWQiOiAxMTMwMjQsICJ0aW1lIjogMTU3NTE1ODQyMywgImhlYXJ0cmF0ZSI6IDk0LjcyNTkyNjkxMDg1MjAxfQ==,bpm,4,65002565,1575158432217,2019-12-01,2019-48
MTQxNjg3,eyJkZXZpY2VfaWQiOiAxNDE2ODcsICJ0aW1lIjogMTU3NTE1ODQzMCwgImhlYXJ0cmF0ZSI6IDg0LjM4NzM2MzY4NjEzMDU0fQ==,bpm,2,65102289,1575158437927,2019-12-01,2019-48
MTQyODM4,eyJkZXZpY2VfaWQiOiAxNDI4MzgsICJ0aW1lIjogMTU3NTE1ODQzNCwgImhlYXJ0cmF0ZSI6IDg1Ljc2MzkzNjgwMDQ1MTAzfQ==,bpm,3,64899162,1575158445876,2019-12-01,2019-48
MTk3OTMw,eyJkZXZpY2VfaWQiOiAxOTc5MzAsICJ0aW1lIjogMTU3NTE1ODQ1MCwgImhlYXJ0cmF0ZSI6IDg0LjYyMDk2MzIyMTczNTA3fQ==,bpm,0,65087190,1575158460427,2019-12-01,2019-48
MTEwMzY3,eyJkZXZpY2VfaWQiOiAxMTAzNjcsICJ0aW1lIjogMTU3NTE1ODQ3MSwgImhlYXJ0cmF0ZSI6IDc2LjUwOTk5NTg3NjA2Njc0fQ==,bpm,2,65102354,1575158482354,2019-12-01,2019-48


In [0]:
batch_df.count()

Out[4]: 879691

#Initial tests
<br>
Initial tests to work out how to clean the data. <br>
Since the raw data has two binary columns, I have to decode it first to move to the next step

In [0]:
display(batch_df.withColumn("key_decoded",batch_df.key.cast('string')) \
        .withColumn("value_decoded",batch_df.value.cast('string')) \
        .select('key','key_decoded','value','value_decoded')\
        .limit(10))

key,key_decoded,value,value_decoded
MTE5NzE1,119715,eyJkZXZpY2VfaWQiOiAxMTk3MTUsICJ0aW1lIjogMTU3NTE1ODQwMCwgImhlYXJ0cmF0ZSI6IDY2LjYyMjQ0OTczMTU3OTc2fQ==,"{""device_id"": 119715, ""time"": 1575158400, ""heartrate"": 66.62244973157976}"
MTA3OTUy,107952,eyJkZXZpY2VfaWQiOiAxMDc5NTIsICJ0aW1lIjogMTU3NTE1ODQwMSwgImhlYXJ0cmF0ZSI6IDExNi4wMzEyNTg5ODAxMjA4N30=,"{""device_id"": 107952, ""time"": 1575158401, ""heartrate"": 116.03125898012087}"
MTYxOTEw,161910,eyJkZXZpY2VfaWQiOiAxNjE5MTAsICJ0aW1lIjogMTU3NTE1ODQwMywgImhlYXJ0cmF0ZSI6IDgyLjMzMzg1OTEyMDg3MzE3fQ==,"{""device_id"": 161910, ""time"": 1575158403, ""heartrate"": 82.33385912087317}"
MTE2NzA4,116708,eyJkZXZpY2VfaWQiOiAxMTY3MDgsICJ0aW1lIjogMTU3NTE1ODQxMiwgImhlYXJ0cmF0ZSI6IDgzLjEzNDUyNzA2MDg3NzA5fQ==,"{""device_id"": 116708, ""time"": 1575158412, ""heartrate"": 83.13452706087709}"
MTYxMzQz,161343,eyJkZXZpY2VfaWQiOiAxNjEzNDMsICJ0aW1lIjogMTU3NTE1ODQyMCwgImhlYXJ0cmF0ZSI6IDU5LjcxODMyNzY1MTYyODA0fQ==,"{""device_id"": 161343, ""time"": 1575158420, ""heartrate"": 59.71832765162804}"
MTEzMDI0,113024,eyJkZXZpY2VfaWQiOiAxMTMwMjQsICJ0aW1lIjogMTU3NTE1ODQyMywgImhlYXJ0cmF0ZSI6IDk0LjcyNTkyNjkxMDg1MjAxfQ==,"{""device_id"": 113024, ""time"": 1575158423, ""heartrate"": 94.72592691085201}"
MTQxNjg3,141687,eyJkZXZpY2VfaWQiOiAxNDE2ODcsICJ0aW1lIjogMTU3NTE1ODQzMCwgImhlYXJ0cmF0ZSI6IDg0LjM4NzM2MzY4NjEzMDU0fQ==,"{""device_id"": 141687, ""time"": 1575158430, ""heartrate"": 84.38736368613054}"
MTQyODM4,142838,eyJkZXZpY2VfaWQiOiAxNDI4MzgsICJ0aW1lIjogMTU3NTE1ODQzNCwgImhlYXJ0cmF0ZSI6IDg1Ljc2MzkzNjgwMDQ1MTAzfQ==,"{""device_id"": 142838, ""time"": 1575158434, ""heartrate"": 85.76393680045103}"
MTk3OTMw,197930,eyJkZXZpY2VfaWQiOiAxOTc5MzAsICJ0aW1lIjogMTU3NTE1ODQ1MCwgImhlYXJ0cmF0ZSI6IDg0LjYyMDk2MzIyMTczNTA3fQ==,"{""device_id"": 197930, ""time"": 1575158450, ""heartrate"": 84.62096322173507}"
MTEwMzY3,110367,eyJkZXZpY2VfaWQiOiAxMTAzNjcsICJ0aW1lIjogMTU3NTE1ODQ3MSwgImhlYXJ0cmF0ZSI6IDc2LjUwOTk5NTg3NjA2Njc0fQ==,"{""device_id"": 110367, ""time"": 1575158471, ""heartrate"": 76.50999587606674}"


###Reviewing decoded data


Keeping in mind that the raw data have three different topics, I'll first ensure that the decoded information is homogeneous between the same topic

In [0]:
topics = ('workout','bpm','user_info')

for i in topics:
        display(batch_df.filter(batch_df.topic == i)
                        .withColumn("key_decoded",batch_df.key.cast('string'))
                        .withColumn("value_decoded",batch_df.value.cast('string'))
                        .select('topic','key_decoded','value_decoded')
                        .limit(5)
              )


topic,key_decoded,value_decoded
workout,40872,"{""user_id"": 40872, ""workout_id"": 8, ""timestamp"": 1575229675.0, ""action"": ""start"", ""session_id"": 76}"
workout,29213,"{""user_id"": 29213, ""workout_id"": 13, ""timestamp"": 1575196562.0, ""action"": ""start"", ""session_id"": 295}"
workout,29213,"{""user_id"": 29213, ""workout_id"": 13, ""timestamp"": 1575200016.0, ""action"": ""stop"", ""session_id"": 295}"
workout,27703,"{""user_id"": 27703, ""workout_id"": 47, ""timestamp"": 1575203433.0, ""action"": ""start"", ""session_id"": 456}"
workout,14508,"{""user_id"": 14508, ""workout_id"": 31, ""timestamp"": 1575224965.0, ""action"": ""start"", ""session_id"": 392}"


topic,key_decoded,value_decoded
bpm,119715,"{""device_id"": 119715, ""time"": 1575158400, ""heartrate"": 66.62244973157976}"
bpm,107952,"{""device_id"": 107952, ""time"": 1575158401, ""heartrate"": 116.03125898012087}"
bpm,161910,"{""device_id"": 161910, ""time"": 1575158403, ""heartrate"": 82.33385912087317}"
bpm,116708,"{""device_id"": 116708, ""time"": 1575158412, ""heartrate"": 83.13452706087709}"
bpm,161343,"{""device_id"": 161343, ""time"": 1575158420, ""heartrate"": 59.71832765162804}"


topic,key_decoded,value_decoded
user_info,27306,"{""user_id"": 27306, ""update_type"": ""new"", ""timestamp"": 1557184117, ""dob"": ""07/16/1958"", ""sex"": ""F"", ""gender"": ""F"", ""first_name"": ""Stephanie"", ""last_name"": ""Martinez"", ""address"": {""street_address"": ""1767 Katherine Expressway Suite 062"", ""city"": ""Edwards"", ""state"": ""CA"", ""zip"": 93523}}"
user_info,31362,"{""user_id"": 31362, ""update_type"": ""new"", ""timestamp"": 1555539598, ""dob"": ""02/01/2001"", ""sex"": ""F"", ""gender"": ""F"", ""first_name"": ""Shelley"", ""last_name"": ""Andrews"", ""address"": {""street_address"": ""4791 Nathan Turnpike Suite 540"", ""city"": ""Los Angeles"", ""state"": ""CA"", ""zip"": 90024}}"
user_info,27703,"{""user_id"": 27703, ""update_type"": ""new"", ""timestamp"": 1557989967, ""dob"": ""11/19/1944"", ""sex"": ""M"", ""gender"": ""M"", ""first_name"": ""Joshua"", ""last_name"": ""Mitchell"", ""address"": {""street_address"": ""843 Hannah Corners"", ""city"": ""Woodland Hills"", ""state"": ""CA"", ""zip"": 91367}}"
user_info,42794,"{""user_id"": 42794, ""update_type"": ""new"", ""timestamp"": 1558904984, ""dob"": ""05/11/1988"", ""sex"": ""M"", ""gender"": ""M"", ""first_name"": ""Brian"", ""last_name"": ""Williams"", ""address"": {""street_address"": ""27880 Dawn Lodge"", ""city"": ""Paramount"", ""state"": ""CA"", ""zip"": 90723}}"
user_info,12140,"{""user_id"": 12140, ""update_type"": ""new"", ""timestamp"": 1558694443, ""dob"": ""02/02/1999"", ""sex"": ""M"", ""gender"": ""M"", ""first_name"": ""Robert"", ""last_name"": ""Clark"", ""address"": {""street_address"": ""68994 Steven Vista"", ""city"": ""Pearblossom"", ""state"": ""CA"", ""zip"": 93553}}"


In [0]:
for i in topics:
        count = batch_df.filter(batch_df.topic == i).count()
        print(f"number of {i} rows = {count}")

number of workout rows = 152
number of bpm rows = 879493
number of user_info rows = 46


Upon first inspection, after decoding: 
<br>
- The data don't need further cleaning
- Key column represents the topic ID
- All key information is on the value column
<br>
<br>
I'm ready to write the decoded data into the main dataframe and begin the work in each topic

In [0]:
batch_df = batch_df.withColumn("key",batch_df.key.cast('string')) \
                   .withColumn("value",batch_df.value.cast('string'))\
                   .select('key', 'value', 'topic', 'partition', 'offset', 'timestamp', 'date', 'week_part')

# Processing the Heart Rate table

In [0]:
batch_df_bpm = batch_df.where(batch_df.topic == 'bpm')

#####Working out how to tranform the json column into a table and the data types of each new column

In [0]:
display(batch_df_bpm.limit(10))

key,value,topic,partition,offset,timestamp,date,week_part
119715,"{""device_id"": 119715, ""time"": 1575158400, ""heartrate"": 66.62244973157976}",bpm,0,65087128,1575158410818,2019-12-01,2019-48
107952,"{""device_id"": 107952, ""time"": 1575158401, ""heartrate"": 116.03125898012087}",bpm,2,65102257,1575158411394,2019-12-01,2019-48
161910,"{""device_id"": 161910, ""time"": 1575158403, ""heartrate"": 82.33385912087317}",bpm,0,65087131,1575158417678,2019-12-01,2019-48
116708,"{""device_id"": 116708, ""time"": 1575158412, ""heartrate"": 83.13452706087709}",bpm,3,64899138,1575158421868,2019-12-01,2019-48
161343,"{""device_id"": 161343, ""time"": 1575158420, ""heartrate"": 59.71832765162804}",bpm,3,64899148,1575158430841,2019-12-01,2019-48
113024,"{""device_id"": 113024, ""time"": 1575158423, ""heartrate"": 94.72592691085201}",bpm,4,65002565,1575158432217,2019-12-01,2019-48
141687,"{""device_id"": 141687, ""time"": 1575158430, ""heartrate"": 84.38736368613054}",bpm,2,65102289,1575158437927,2019-12-01,2019-48
142838,"{""device_id"": 142838, ""time"": 1575158434, ""heartrate"": 85.76393680045103}",bpm,3,64899162,1575158445876,2019-12-01,2019-48
197930,"{""device_id"": 197930, ""time"": 1575158450, ""heartrate"": 84.62096322173507}",bpm,0,65087190,1575158460427,2019-12-01,2019-48
110367,"{""device_id"": 110367, ""time"": 1575158471, ""heartrate"": 76.50999587606674}",bpm,2,65102354,1575158482354,2019-12-01,2019-48


In [0]:
schema = t.StructType(
    [
        t.StructField('device_id', t.IntegerType(), True),
        t.StructField('time', t.TimestampType(), True),
        t.StructField('heartrate', t.DoubleType(), True)
    ]
)

bpm_df = batch_df_bpm.withColumn('value', f.from_json('value', schema))\
                     .select(f.col('value.*'))

display(bpm_df.limit(10))

device_id,time,heartrate
163871,2019-12-02T15:40:59.000+0000,81.52409462853655
102558,2019-12-02T15:41:01.000+0000,94.45364094154255
161343,2019-12-02T15:41:00.000+0000,66.72305029774591
135279,2019-12-02T15:41:01.000+0000,90.95946373662196
124880,2019-12-02T15:41:11.000+0000,76.33347856472393
118440,2019-12-02T15:41:18.000+0000,81.64002273572957
102558,2019-12-02T15:41:20.000+0000,85.17778434335672
197930,2019-12-02T15:41:22.000+0000,85.95268901953999
107952,2019-12-02T15:41:32.000+0000,107.37487817461707
116708,2019-12-02T15:41:21.000+0000,89.36147934287175


#####Checking for duplicate rows

In [0]:
bpm_df.dropDuplicates().count()

Out[12]: 731987

In [0]:
bpm_df.dropDuplicates(["device_id", "time"]).count()

Out[13]: 731987

Since all the duplicates are totally equal rows, there is no need to inspect this aspect further

In [0]:
bpm_df = bpm_df.dropDuplicates()

#####Checking for negative values
<br>
Since it's impossible to have a negative heart rate, it's a best practice o check

In [0]:
display(bpm_df.where(f.col('heartrate') < 0).limit(10))

device_id,time,heartrate
113570,2019-12-02T17:09:20.000+0000,-93.98990072277059
116708,2019-12-02T19:37:21.000+0000,-79.27196704487537
144334,2019-12-02T12:45:45.000+0000,-103.67377032221478
110367,2019-12-02T05:14:20.000+0000,-75.64529924157084
180111,2019-12-02T08:47:33.000+0000,-62.35650506300333
187047,2019-12-02T02:14:20.000+0000,-42.68500952715515
193806,2019-12-02T08:53:23.000+0000,-74.30224557940427
148312,2019-12-02T23:32:32.000+0000,-49.494806454842326
144334,2019-12-02T22:52:13.000+0000,-60.6433180748106
187047,2019-12-02T21:26:00.000+0000,-80.60716366472212


In [0]:
display(bpm_df.where(f.col('heartrate') < 0).count())

241

Since there is so few negative values, it isn't worth treating them. The way I chose to deal with them is by table constraints

#####Creating the Silver layer for the new table


In [0]:
%sql
create schema if not exists silver

#####Compiling the code above and including it to the streaming logic

Using the **`trigger(availableNow=True)`** option will process all records (in multiple batches if needed) until no more data is available and then stop the stream.

In [0]:
json_schema = "device_id INTEGER, time TIMESTAMP, heartrate DOUBLE"
checkpoint_path = f'{main_path}/_checkpoints'

def process_heart_rate():
    query = (spark.readStream
                    .table("bronze")
                    .filter("topic = 'bpm'")
                    .select(f.from_json(f.col("value").cast("string"), json_schema).alias("bpm"))
                    .filter("bpm.heartrate > 0")
                    .select("bpm.*")
                    .withWatermark("time", "30 seconds")
                    .dropDuplicates()
                    .writeStream
                        .option("checkpointLocation", f"{checkpoint_path}/heart_rate")
                        .trigger(availableNow=True)
                        .table("silver.heart_rate"))

    query.awaitTermination()

In [0]:
%sql
drop table if exists silver.heart_rate

In [0]:
dbutils.fs.rm("dbfs:/mnt/borges_portifolio/_checkpoints/heart_rate", True)

Out[20]: False

In [0]:
process_heart_rate()

In [0]:
%sql
ALTER TABLE silver.heart_rate ADD CONSTRAINT validbpm CHECK (heartrate > 0);

#####Assuring that everything went as planned

In [0]:
%sql

describe extended silver.heart_rate

col_name,data_type,comment
device_id,int,null
time,timestamp,null
heartrate,double,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,silver,
Table,heart_rate,
Created Time,Fri May 03 12:08:40 UTC 2024,
Last Access,UNKNOWN,


In [0]:
%sql
select count(*) from silver.heart_rate

count(1)
731746


In [0]:
%sql
select * from silver.heart_rate a limit 10

device_id,time,heartrate
102558,2019-12-02T15:58:20.000+0000,90.74234303723293
107952,2019-12-02T16:16:50.000+0000,99.92202226593994
105671,2019-12-02T16:24:41.000+0000,92.3830462993785
135279,2019-12-02T17:00:32.000+0000,81.08301232974536
102558,2019-12-02T17:06:20.000+0000,89.19526024529921
109290,2019-12-02T17:35:04.000+0000,64.56235930836219
158496,2019-12-02T17:37:22.000+0000,89.13700402219997
196248,2019-12-02T17:48:52.000+0000,120.019696200589
113570,2019-12-02T18:07:23.000+0000,85.39573128945582
196248,2019-12-02T18:09:44.000+0000,114.52080986523916


# Processing the Workout table

In [0]:
batch_df_bpm = batch_df.where(batch_df.topic == 'bpm')

In [0]:
display(batch_df_bpm.limit(10))

In [0]:
schema = t.StructType(
    [
        t.StructField('device_id', t.IntegerType(), True),
        t.StructField('time', t.TimestampType(), True),
        t.StructField('heartrate', t.DoubleType(), True)
    ]
)

bpm_df = batch_df_bpm.withColumn('value', f.from_json('value', schema))\
                     .select(f.col('value.*'))

display(bpm_df.limit(10))

In [0]:
def execute_stream():
        (spark.readStream
              .option("ignoreDeletes", True)
              .table("bronze")
              .filter("topic = 'workout'")
              .select(F.from_json(F.col("value").cast("string"), "user_id INT, workout_id INT, timestamp FLOAT, action STRING, session_id INT").alias("v"))
              .select("v.*")
              .select("user_id", "workout_id", F.col("timestamp").cast("timestamp").alias("time"), "action", "session_id")
              .withWatermark("time", "30 seconds")
              .dropDuplicates(["user_id", "time"])
              .writeStream
              .foreachBatch(streamingMerge.upsertToDelta)
              .outputMode("update")
              .option("checkpointLocation", f"{DA.paths.checkpoints}/workouts")
              .queryName("workouts_silver")
              .trigger(availableNow=True)
              .start()
              .awaitTermination())